# ایزور اے آئی ایجنٹس کے ساتھ ماڈل کانٹیکسٹ پروٹوکول (MCP) کی سپورٹ

یہ نوٹ بک دکھاتی ہے کہ ایزور اے آئی ایجنٹس کو ماڈل کانٹیکسٹ پروٹوکول (MCP) ٹولز کے ساتھ کیسے استعمال کیا جا سکتا ہے تاکہ ایک ذہین ایجنٹ بنایا جا سکے جو اضافی صلاحیتوں کے لیے بیرونی MCP سرورز کا فائدہ اٹھا سکے۔


## مطلوبہ NuGet پیکجز انسٹال کریں

سب سے پہلے، ہمیں Azure AI Agents Persistent پیکج انسٹال کرنا ہوگا جو Azure AI Agents کے ساتھ کام کرنے کے لیے بنیادی فعالیت فراہم کرتا ہے۔


## کلید کے بغیر تصدیق کے فوائد

یہ نوٹ بک **کلید کے بغیر تصدیق** کو ظاہر کرتی ہے جو کئی فوائد فراہم کرتی ہے:
- ✅ **کوئی API کلیدیں سنبھالنے کی ضرورت نہیں** - Azure کی شناخت پر مبنی تصدیق استعمال کرتی ہے
- ✅ **بہتر سیکیورٹی** - کوڈ یا کنفیگریشن میں کوئی راز محفوظ نہیں کیے جاتے
- ✅ **خودکار اسناد کی گردش** - Azure اسناد کے زندگی کے دور کو سنبھالتا ہے
- ✅ **کردار پر مبنی رسائی** - Azure RBAC کے ذریعے باریک بینی سے اجازتیں فراہم کرتا ہے

`DefaultAzureCredential` خود بخود بہترین دستیاب اسناد کے ذریعہ استعمال کرے گا:
1. منیجڈ شناخت (جب Azure میں چل رہا ہو)
2. Azure CLI اسناد (ترقی کے دوران)
3. Visual Studio اسناد
4. ماحول کے متغیرات (اگر ترتیب دیے گئے ہوں)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

ایزور سروسز کے ساتھ تصدیق کے لیے ڈیفالٹ ایزور کریڈینشل استعمال کرتے ہوئے ایزور آئیڈینٹی پیکیج انسٹال کریں۔


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## ضروری نامسپیسز درآمد کریں

Azure AI Agents اور Azure Identity کے لیے ضروری نامسپیسز درآمد کریں۔


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## ایزور اے آئی ایجنٹ کلائنٹ کو ترتیب دیں (کی لیس تصدیق)

ترتیب کے متغیرات سیٹ کریں اور PersistentAgentsClient کو **کی لیس تصدیق** کے ذریعے بنائیں:
- **projectEndpoint**: ایزور اے آئی فاؤنڈری پروجیکٹ کا اینڈ پوائنٹ
- **modelDeploymentName**: تعینات کردہ اے آئی ماڈل کا نام (GPT-4.1 نینو)
- **mcpServerUrl**: ایم سی پی سرور کا یو آر ایل (مائیکروسافٹ لرن اے پی آئی)
- **mcpServerLabel**: ایم سی پی سرور کی شناخت کے لیے ایک لیبل
- **DefaultAzureCredential**: مینیجڈ آئیڈینٹیٹی، ایزور سی ایل آئی، یا دیگر تصدیقی ذرائع استعمال کرتا ہے (کسی اے پی آئی کیز کی ضرورت نہیں)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## ایم سی پی ٹول کی تعریف بنائیں

ایم سی پی ٹول کی تعریف بنائیں جو مائیکروسافٹ لرن ایم سی پی سرور سے جڑتی ہو۔ یہ ایجنٹ کو مائیکروسافٹ لرن کے مواد اور دستاویزات تک رسائی فراہم کرے گا۔


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## اے آئی ایجنٹ بنائیں

ایک مستقل اے آئی ایجنٹ بنائیں جو دیے گئے ماڈل اور ایم سی پی ٹولز کے ساتھ کام کرے۔ ایجنٹ کو درج ذیل کے ساتھ ترتیب دیا گیا ہے:
- جی پی ٹی-4.1 نینو ماڈل
- ایم سی پی ٹولز کے استعمال کے لیے ہدایات
- مائیکروسافٹ لرن ایم سی پی سرور تک رسائی


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## تھریڈ بنائیں اور پیغام بھیجیں

ایک گفتگو کا تھریڈ بنائیں اور صارف کو ایک پیغام بھیجیں جس میں Azure OpenAI اور OpenAI کے درمیان فرق کے بارے میں پوچھا جائے۔ یہ ایجنٹ کی صلاحیت کو MCP ٹولز استعمال کرنے اور درست معلومات فراہم کرنے کی جانچ کرے گا۔


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## ایم سی پی ٹول کے وسائل کو ترتیب دیں (بغیر کلید کے)

ایم سی پی ٹول کے وسائل کو ترتیب دیں۔ ایک مکمل طور پر بغیر کلید کے طریقہ کار کے لیے، آپ کسٹم ہیڈرز کو ہٹا سکتے ہیں اگر ایم سی پی سرور Azure شناخت پر مبنی تصدیق کی حمایت کرتا ہو۔ نیچے دی گئی مثال میں دکھایا گیا ہے کہ اگر ضرورت ہو تو ہیڈرز کیسے شامل کیے جائیں، لیکن بغیر کلید کے منظرناموں کے لیے، یہ ضروری نہیں ہو سکتے۔


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## ایجنٹ رن شروع کریں

صارف کے پیغام کو پروسیس کرنے کے لیے ایک رن بنائیں اور شروع کریں۔ ایجنٹ ترتیب دیے گئے MCP ٹولز اور وسائل کا استعمال کرتے ہوئے جواب تیار کرے گا۔


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## ایجنٹ کی کارکردگی کی نگرانی کریں اور ٹول کی منظوریوں کو سنبھالیں (بغیر کلید کے)

ایجنٹ کی کارکردگی کی حالت کی نگرانی کریں اور کسی بھی ضروری ٹول کی منظوریوں کو سنبھالیں۔ یہ عمل:
1. کارکردگی مکمل ہونے یا کارروائی کی ضرورت ہونے کا انتظار کرتا ہے
2. جب ضرورت ہو تو MCP ٹول کالز کو خودکار طور پر منظور کرتا ہے
3. بغیر کلید کی تصدیق کے لیے، اگر MCP سرور Azure شناخت کی حمایت کرتا ہے تو ہیڈرز کی ضرورت نہیں ہو سکتی


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## گفتگو کے نتائج دکھائیں

تھریڈ میں موجود تمام پیغامات کو حاصل کریں اور دکھائیں، جس میں صارف کے سوال اور ایجنٹ کے جواب دونوں شامل ہوں۔ پیغامات کو وقت کے مطابق ترتیب میں ٹائم اسٹیمپ اور کردار کے اشارے کے ساتھ دکھایا جاتا ہے۔


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
